In [ ]:
using Pkg
Pkg.activate("../..")
Pkg.instantiate()

# Batagelj-Zaveršnik k-core decomposition algorithm

In [ ]:
using Graphs

function batagelj_zaversnik_k_core(g::SimpleGraph)
  n_v = nv(g)
  D = zeros(Int64, n_v)
  pos = zeros(Int64, n_v)
  vert = zeros(Int64, n_v)
  md = 0
  for v in 1:n_v
    D[v] = degree(g, v)
    md = max(md, D[v])
  end

  bin = zeros(Int64, md)

  for v in 1:n_v
    bin[D[v]] += 1
  end

  start = 1
  for d in 1:md
    num = bin[d]
    bin[d] = start
    start += num
  end

  for v in 1:n_v
    pos[v] = bin[D[v]]
    vert[pos[v]] = v
    bin[D[v]] += 1
  end

  for d in md:-1:2
    bin[d] = bin[d-1]
  end
  bin[1] = 1

  for i in 1:n_v
    v = vert[i]
    for u in neighbors(g, v)
      if D[u] > D[v]
        du = D[u]
        pu = pos[u]
        pw = bin[du]
        w = vert[pw]
        if u != w
          pos[u] = pw
          pos[w] = pu
          vert[pu] = w
          vert[pw] = u
        end
        bin[du] += 1
        D[u] -= 1
      end
    end
  end

  D
end

In [ ]:
using Graphs

function batagelj_zaversnik_k_core_as_network(g::SimpleGraph, k::Int64)
  indices = batagelj_zaversnik_k_core(g)
  # create a vector of vertices that are in the k-core
  induction_vertices = [i for i in 1:length(indices) if indices[i] >= k]
  # create a subgraph of the original graph with only the vertices in the k-core
  induced_subgraph(g, induction_vertices)
end

In [ ]:
using Graphs

function vmap_old_to_new(l::Int64, vmap::Vector{Int64})
  vmap_old_to_new = zeros(Int64, l)
  for i in 1:length(vmap)
    vmap_old_to_new[vmap[i]] = i
  end
  vmap_old_to_new
end

function straight_forward_k_core(g::SimpleGraph, k::Int64)
  if nv(g) == 0
    return g
  end

  # copy the graph
  g2 = SimpleGraph()
  for i in 1:nv(g)
    add_vertex!(g2)
  end
  for e in edges(g)
    add_edge!(g2, e.src, e.dst)
  end

  # ε is the list of vertices in the k-core
  ε = vertices(g2)
  # remove vertices with degree greater than k
  while true
    achieved_k_core = true

    # vertices that will be removed
    δ = Vector{Int64}()
    for i in 1:nv(g2)
      if degree(g2, i) < k
        achieved_k_core = false
        push!(δ, i)
      end
    end
    if achieved_k_core
      break
    end

    # remove vertices from δ in ε
    ε = [x for x in ε if !(x in δ)]

    l = nv(g2)
    g2, vmap = induced_subgraph(g2, ε)
    vmap = vmap_old_to_new(l, vmap)
    ε = map(x -> vmap[x], ε)
  end

  g2
end

In [ ]:
using Graphs

g = barabasi_albert_simple_graph(100, 0.1, 200, 5)
@show g
@show straight_forward_k_core(g, 6)
@show batagelj_zaversnik_k_core_as_network(g, 6)


In [ ]:
using Graphs

g = barabasi_albert_simple_graph(100, 0.1, 200, 5)

g2, vmap = batagelj_zaversnik_k_core_as_network(g, 5)

@info nv(g)
@info nv(g2)

# Simple test graphs (15-20 nodes)

Graphs are generated by a human. They are hardcoded here.

In [ ]:
using Graphs

small_graph_1 = SimpleGraph()

for i in 1:18
  add_vertex!(small_graph_1)
end

edge_pairs_1 = [
  (1, 2),
  (1, 3),
  (2, 3),
  (2, 8),
  (4, 7),
  (4, 8),
  (5, 6),
  (6, 7),
  (7, 8),
  (7, 10),
  (7, 11),
  (7, 12),
  (7, 13),
  (7, 15),
  (8, 9),
  (8, 10),
  (8, 12),
  (9, 16),
  (12, 14),
  (14, 18),
  (16, 17),
]

for (i, j) in edge_pairs_1
  add_edge!(small_graph_1, i, j)
end

println(k_core(small_graph_1))
println(batagelj_zaversnik_k_core(small_graph_1))

Since shell index 2 is the largest one in this graph, by comparing the result of the implemented algorithm with the Graphs.jl implementation we can conclude that the implementation is correct.

In [ ]:
using Graphs

small_graph_2 = SimpleGraph()

for i in 1:17
  add_vertex!(small_graph_2)
end

edge_pairs_2 = [
  (1, 2),
  (1, 3),
  (2, 3),
  (3, 4),
  (4, 7),
  (4, 8),
  (4, 10),
  (5, 11),
  (6, 7),
  (7, 8),
  (7, 9),
  (7, 10),
  (7, 14),
  (7, 16),
  (8, 10),
  (8, 11),
  (9, 13),
  (10, 15),
  (10, 17),
  (11, 12),
]

for (i, j) in edge_pairs_2
  add_edge!(small_graph_2, i, j)
end

println(k_core(small_graph_2))
println(batagelj_zaversnik_k_core(small_graph_2))

In [ ]:
using Graphs

small_graph_3 = SimpleGraph()

for i in 1:23
  add_vertex!(small_graph_3)
end

egde_pairs_3 = [
  (1, 7),
  (2, 7),
  (3, 7),
  (3, 8),
  (3, 12),
  (4, 5),
  (5, 9),
  (6, 7),
  (6, 12),
  (7, 12),
  (8, 12),
  (9, 13),
  (10, 11),
  (11, 12),
  (11, 15),
  (11, 16),
  (12, 13),
  (12, 16),
  (12, 17),
  (12, 18),
  (13, 14),
  (13, 18),
  (15, 19),
  (15, 20),
  (16, 21),
  (17, 21),
  (18, 22),
  (18, 23),
]

for (i, j) in egde_pairs_3
  add_edge!(small_graph_3, i, j)
end

println(k_core(small_graph_3))
println(batagelj_zaversnik_k_core(small_graph_3))

In [ ]:
using Graphs

large_graph_1 = SimpleGraph(100, 1650)

println(k_core(large_graph_1))
println(batagelj_zaversnik_k_core(large_graph_1))

In [ ]:
using Graphs
large_graph_2 = SimpleGraph(1000, 166500)

println(k_core(large_graph_2))
println(batagelj_zaversnik_k_core(large_graph_2))

In [ ]:
rand(1:10, 5)

In [ ]:
function erdos_renyi_simple_graph(vs::Int64, es::Int64)
  S = Iterators.product(1:vs, 1:vs) |> collect |> filter(x -> x[1] != x[2])
  Random.shuffle!(S)

  G = SimpleGraph(vs)

  for (i, j) in S[1:es]
    add_edge!(G, i, j)
  end

  G
end

In [ ]:
using Graphs

erdos_renyi_graph_1 = erdos_renyi_simple_graph(10000, 165000)

println(k_core(erdos_renyi_graph_1))
println(batagelj_zaversnik_k_core(erdos_renyi_graph_1))

In [ ]:
using Graphs

function gilbert_simple_graph(vs::Int64, p::Float64)
  G = SimpleGraph(vs)

  for i in 1:vs-1
    for j in i+1:vs
      if rand() < p
        add_edge!(G, i, j)
      end
    end
  end
  
  G
end

In [ ]:
using Graphs

gilbert_graph_1 = gilbert_simple_graph(10000, 0.0165)

println(k_core(gilbert_graph_1))
println(batagelj_zaversnik_k_core(gilbert_graph_1))

In [ ]:
using Graphs

function barabasi_albert_simple_graph(m0::Int64, p::Float64, N::Int64, m::Int64)
  G = gilbert_simple_graph(m0, p)

  Δ = []
  for i in 1:m0
    append!(Δ, repeat([i], degree(G, i)))
  end

  for i in m0+1:N
    add_vertex!(G)

    for j in 1:m
      v = rand(Δ)
      add_edge!(G, i, v)
      append!(Δ, [v])
    end

    append!(Δ, repeat([i], m))
  end

  G
end

In [ ]:
using Graphs

barabasi_albert_graph_1 = barabasi_albert_simple_graph(100, 0.1, 200, 5)

println(k_core(barabasi_albert_graph_1))
println(batagelj_zaversnik_k_core(barabasi_albert_graph_1))
println(core_periphery_deg(barabasi_albert_graph_1))

@drawsvg begin
  background("black")
  sethue("grey40")
  fontsize(8)
  drawgraph(barabasi_albert_graph_1,
    layout=stress,
    vertexlabels = vertices(barabasi_albert_graph_1),
    vertexfillcolors = 
      [RGB(rand(3)/2...)
        for i in vertices(barabasi_albert_graph_1)],
  )
end

In [ ]:
using Graphs

function stochastic_block_model_simple_graph(N::Int64, q::Int64, m::Vector{Float64}, B::Matrix{Float64})
  G = gilbert_simple_graph(N, 0.005)
  g = Vector{Int64}(undef, N)

  for i in 1:N
    r = rand()
    s = 0.0
    for k in 1:q
      s += m[k]
      if r <= s
        g[i] = k
        break
      end
    end
  end

  @show g

  for i in 1:N-1
    for j in i+1:N
      if rand() <= B[g[i], g[j]]
        add_edge!(G, i, j)
      end
    end
  end

  G
end

In [ ]:
using Graphs
using Karnak
using NetworkLayout
using Colors

stochastic_block_model_graph_1 = stochastic_block_model_simple_graph(100, 3, [0.33, 0.33, 0.34], [0.8 0.1 0.1; 0.1 0.8 0.1; 0.1 0.1 0.8])

@drawsvg begin
  background("black")
  sethue("grey40")
  fontsize(8)
  drawgraph(stochastic_block_model_graph_1,
    layout=stress,
    vertexlabels = vertices(stochastic_block_model_graph_1),
    vertexfillcolors = 
      [RGB(rand(3)/2...)
        for i in vertices(stochastic_block_model_graph_1)]
  )
end